first we need to ensure that this library is installed

In [ ]:
!pip install --upgrade openai

in this part of code we set up the variable needed to complete all the code:

- file_path: the path of the file we are reading
- output_filename: the name we want for the output file
- key: the OpenAi token
- sleep_time: the time that our code will wait to check for an answer

In [ ]:
import time
import requests
import base64
from openai import OpenAI

# === CONFIG ===
file_path = "/content/invented_news_articles.txt"

output_filename="summary_articles.html"

key = "yourkeyhere"

client = OpenAI(api_key=key)


this function create:
- Assistant: it's like giving the AI the personality we want and that are memorized only once
- Thread: it's like a chat, we have history, and we can interact like if we were chatting with a friends, the difference with a normal call is that we have history so we can say now that you did X do Y

In [ ]:
# === CREATE ASSISTANT AND THREAD ===
def setup_assistant():
    assistant = client.beta.assistants.create(
        name="NewsToImageAssistant", # just the name of the assistant
        # instruction want a string and is the job / personality of the assistant
        instructions=(
            "You are an AI assistant. First, you receive a text to summarize, then you transform that summary into a detailed visual prompt for an illustrated-style image (DALL·E). Maintain a positive, clear, and visually inspired tone."
        ),
        model="gpt-4-turbo" # model we want to use for this assistant (every message will use that)
    ) # this function return the id of the assistant created with the specified characteristics

    thread = client.beta.threads.create() # this function return a string that is the id of the thread

    return assistant.id, thread.id # returning the 2 strings


this function just read the file, split it on the ----- an returns a list of string, every string contains an article


In [ ]:
# === READ ARTICLES ===
def read_articles_from_file():
    try:
        # lets try to open the file, given the file path
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            # Using '-----' as separator
            articles = [article.strip() for article in content.split('-----') if article.strip()]
            return articles
    except Exception as e:
        print(f"❌ Error while reading the file: {e}")
        return []

this function allow us to send a message to the ai
we give the personality (assistant) and the conversation we want to use (thread) and last the message we want to pass.

after that we create the message saying that our role is the user and we send the message

now we wait until the run (work done by the AI) is completed and we get back the message

after that we just return the content of the work (the structure can be bigger and contain different information but we want only the message)

In [ ]:
# === SEND THE MESSAGE TO THREAD AND GET A REPLY FROM GPT ===
def send_message_to_thread(assistant_id, thread_id, user_message):

    # let's create a message on the thread
    client.beta.threads.messages.create(
        thread_id=thread_id, # the id of the conversation
        role="user", # the role this message has, user = recived by an user
        content=user_message # the message the AI will process
    )

    # Start the run
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    # Wait till completed, we check every x seconds
    while True:
        run_status = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id) # get the status of our message

        #now check the status if not completed or faild it still running
        if run_status.status == "completed":
            break # if the status is completed we stop the loop
        elif run_status.status == "failed":
            print("❌ Run failed OpenAi / Connection side problem.")
            exit() # if the status is failed, print 'run failed' and stop the function

        time.sleep(1) #second to wait before another check

    # Get the response
    messages = client.beta.threads.messages.list(thread_id=thread_id) # save all the messages
    return messages.data[0].content[0].text.value # return the first -> the most recent -> the reply to this prompt


this function send a message to dall-e

In [ ]:
# === CREATE IMAGE WITH DALL·E ===
def generate_image(prompt):
    response = client.images.generate(
        model="dall-e-3", # model we want to use for the generation of the image
        prompt=prompt, # the text to generate the images
        n=1, # number of images to be generated
        size="1024x1024" # dimension of the image
    )
    return response.data[0].url


This function creates the output file

In [ ]:
# === GENERATE OUTPUT FILE ===
def generate_html(data):

    # create the file
    with open(output_filename, "w", encoding="utf-8") as f:
        # we write the first string html line
        f.write("<html><body style='font-family:sans-serif;'>\n")

        # loop every tuple of summary and image url and build the file
        for i, (summary, image_url) in enumerate(data):
            # write the article number as a title
            f.write(f"<h2>Article {i + 1}</h2>\n")
            #just paste the summary of this article
            f.write(f"<p>{summary}</p>\n")

            # if there is an image url we download the image and attach it to the file
            if image_url: # if there is the url of the image
                try:
                    response = requests.get(image_url) # ---- download image
                    response.raise_for_status()
                    img_data = response.content
                    img_base64 = base64.b64encode(img_data).decode("utf-8") # ----
                    f.write(f"<img src='data:image/png;base64,{img_base64}' width='400'><br><br>\n") # add image do the file
                except Exception as e:
                    f.write("<p><i>Image could not be loaded</i></p>\n")
                    print(f"⚠️ Error downloading image {image_url}: {e}")

            f.write("<hr>\n") # line at the end of every the article

        f.write("</body></html>") # end of file
    print(f"\n✅ HTML file saved as: {output_filename}")


this function aim to clean up resources, delete the assistant and the thread

In [ ]:
# === CLEAN UP RESOURCES ===
def cleanup_resources(assistant_id, thread_id):
    """Elimina l'assistente e il thread se non sono più necessari."""
    try:
        client.beta.assistants.delete(assistant_id)
        print(f"\n🧹 Assistant deleted: {assistant_id}")
    except Exception as e:
        print(f"\nError during assistant elimination: {e}")

    try:
        client.beta.threads.delete(thread_id)
        print(f"🧼 Thread deleted: {thread_id}")
    except Exception as e:
        print(f"Error during thread elimination: {e}")


this is the main aprt of the project and it's divi

In [ ]:
def summarize_and_generate(assistant_id, thread_id, article_text):
    # FASE 3: Summary
    user_message = f"Please make a summary of this text:\n\n{article_text}"
    summary = send_message_to_thread(assistant_id, thread_id, user_message)
    print("\n✍️ Summary:\n", summary)

    # FASE 4: Prompt for the image
    user_message = "Generate an engaging, illustrated image for a news article. The scene should clearly reflect the story’s key themes and context, with attention to setting, characters, actions, and symbolic objects. Use a visually clear, editorial illustration style suitable for news publications. Ensure the mood, color palette, and composition support the article’s message. Avoid generic visuals—focus on storytelling through meaningful, contextual detail."
    prompt = send_message_to_thread(assistant_id, thread_id, user_message)
    print("\n🎨 Prompt for DALL·E:\n", prompt)

    # FASE 5: Image
    image_url = generate_image(prompt)
    print("\n🖼️ Generated image:\n", image_url)

    return summary, image_url


# === AVVIO SCRIPT ===
if __name__ == "__main__":

    # FASE 0: Create assistant and thread
    assistant_id, thread_id = setup_assistant()
    print("✅ Assistant e thread created.")

    # FASE 1: Read all the articles
    articles = read_articles_from_file()
    print("\n🖨️ Here what i readed")
    number = 0
    for article in articles:
        print("\nArticle: ", number+1)
        print(article)
        number += 1
    print("\nTotal articles:", number)

    print("\n📝🖌️ Let's start with the summary and the image generation")

    # FASE 2: Get the AI work done
    data_for_the_file = []
    number = 0
    for article in articles:
        print("\n ✉️ Article: ", number+1)
        summary, image_url = summarize_and_generate(assistant_id, thread_id, article)
        data_for_the_file.append((summary, image_url))
        number += 1

    # FASE 6: Create the output file
    generate_html(data_for_the_file)

    # FASE 7: Delete thread and assistant
    cleanup_resources(assistant_id, thread_id)
    print("\n🧽 Resources cleaned up.")


✅ Assistant e thread created.

🖨️ Here what i readed

Article:  1
City Installs Vertical Farms in Subway Tunnels
Abandoned subway lines repurposed to grow fresh produce underground
In a groundbreaking urban agriculture initiative, the city has transformed disused subway tunnels into vertical farms. These underground farms use hydroponic technology and LED lighting to grow leafy greens and herbs year-round.

Officials say the project aims to reduce the carbon footprint of food transport and create local jobs. Early results show promising yields, with plans to expand the project across the metro area by next spring.

Article:  2
Artificial Island for Artists Rises in Baltic Sea
Floating platform offers studios, performance spaces, and isolation
A new artificial island has been constructed off the coast of Estonia to house a community of artists seeking solitude and inspiration. Dubbed 'Artopia', the island features eco-friendly housing, exhibition halls, and outdoor amphitheaters.

The p